In [2]:
import random

import pandas as pd
import numpy as np
import math

#读入数据集
u1_base = pd.read_csv('ml-100k/u1.base', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])
u1_test = pd.read_csv('ml-100k/u1.test', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])

#初始化
user_num = u1_base['uid'].max()
item_num = u1_base['iid'].max()

rating_matrix = np.zeros((user_num, item_num), float)
y_ui = np.zeros((user_num, item_num), int)

#base记录转化为matrix
for index, row in u1_base.iterrows():
    user_id = row['uid']
    item_id = row['iid']
    rating = row['rate']
    rating_matrix[user_id - 1, item_id - 1] = rating
    y_ui[user_id - 1, item_id - 1] = 1

#全局平均
GlobalAverage = rating_matrix.sum() / y_ui.sum()

#计算四个参数 user_means item_means user_bias item_bias
rating_sum_row = [sum(row) for row in rating_matrix]
y_sum_row = [sum(row) for row in y_ui]

rating_sum_col = [sum(column) for column in zip(*rating_matrix)]
y_sum_col = [sum(column) for column in zip(*y_ui)]

user_means = []
for i in range(user_num):
    if y_sum_row[i] == 0:
        user_means.append(GlobalAverage)
    else:
        user_means.append(rating_sum_row[i] / y_sum_row[i])

item_means = []
for i in range(item_num):
    if y_sum_col[i] == 0:
        item_means.append(GlobalAverage)
    else:
        item_means.append(rating_sum_col[i] / y_sum_col[i])

user_bias = []
for i in range(user_num):
    if y_sum_row[i] == 0:
        user_bias.append(0)
    else:
        sum_bias = 0
        for j in range(item_num):
            sum_bias += y_ui[i][j] * (rating_matrix[i][j] - GlobalAverage)
        user_bias.append(sum_bias / y_sum_row[i])

item_bias = []
for i in range(item_num):
    if y_sum_col[i] == 0:
        item_bias.append(0)
    else:
        sum_bias = 0
        for j in range(user_num):
            sum_bias += y_ui[j][i] * (rating_matrix[j][i] - GlobalAverage)
        item_bias.append(sum_bias / y_sum_col[i])


In [2]:
def items_u_r(u, item, r):
    items = np.where(rating_matrix[u] == r)[0]
    items = items[items != item]
    return items


In [3]:
#超参数
lmda = 0.001
lr = 0.01
d = 20
epoch = 20
u1_base_num = len(u1_base)


In [4]:
def U_MPC(u, i, M):
    U_mpc = np.zeros(d, float)
    for rating_class in range(5):
        items = items_u_r(u, i, rating_class + 1)
        items_len = len(items)
        if items_len == 0:
            continue
        U_mpc += np.sum(M[rating_class, items, :], axis=0) / items_len ** 0.5
    return U_mpc

def R_HAT(u, i, miu, U, V, user_bias, item_bias, M):
    return miu + user_bias[u] + item_bias[i] + np.dot(U[u], V[i]) + np.dot(U_MPC(u, i, M), V[i])


In [5]:
U = np.random.rand(user_num + 1, d)
V = np.random.rand(item_num + 1, d)
M = np.random.rand(5, item_num, d)
U = (U - 0.5) * 0.01
V = (V - 0.5) * 0.01
M = (M - 0.5) * 0.01
miu = GlobalAverage
for t in range(epoch):
    indices = list(range(u1_base_num))
    random.shuffle(indices)
    for n in range(u1_base_num):
        print(f'\r{t + 1}:{n + 1}', end='')
        row = u1_base.iloc[indices[n]]
        user_id = row['uid'] - 1
        item_id = row['iid'] - 1
        rating = row['rate']

        U_mpc = U_MPC(user_id, item_id, M)
        e_ui = rating - R_HAT(user_id, item_id, miu, U, V, user_bias, item_bias, M)
        #计算梯度
        delta_miu = -e_ui
        delta_bu = -e_ui + lmda * user_bias[user_id]
        delta_bi = -e_ui + lmda * item_bias[item_id]
        delta_Uu = -e_ui * V[item_id] + lmda * U[user_id]
        delta_Vi = -e_ui * (U[user_id] + U_mpc) + lmda * V[item_id]

        for r in range(5):
            i_pie = items_u_r(user_id, item_id, r + 1)
            i_num = len(i_pie)
            if i_num == 0:
                continue
            for i in i_pie:
                delta_M = -e_ui * V[item_id] / i_num ** 0.5 + lmda * M[r][i]
                M[r][i] -= lr * delta_M
        #update 

        miu -= lr * delta_miu
        user_bias[user_id] -= lr * delta_bu
        item_bias[item_id] -= lr * delta_bi
        U[user_id] -= lr * delta_Uu
        V[item_id] -= lr * delta_Vi
    lr *= 0.9

20:80000

In [14]:
def ERR(test):
    cnt = 0
    abs_err = 0
    squ_err = 0

    for index, row in test.iterrows():
        user_id = row['uid'] - 1
        item_id = row['iid'] - 1
        real_rating = row['rate']

        predicted_rating = R_HAT(user_id, item_id, miu, U, V, user_bias, item_bias, M)
        if predicted_rating > 5:
            predicted_rating = 5
        if predicted_rating < 1:
            predicted_rating = 1

        # 计算绝对误差/平方误差
        absolute_error = abs(predicted_rating - real_rating)
        abs_err += absolute_error

        square_error = pow(predicted_rating - real_rating, 2)
        squ_err += square_error

        cnt += 1
    # 计算平均绝对误差/平方误差
    mae = abs_err / cnt
    rmse = (squ_err / cnt) ** 0.5
    return mae, rmse

In [15]:
mae, rmse = ERR(u1_test)
print(f'(RMSE:{rmse:.4f},MAE:{mae:.4f})')   

(RMSE:0.9175,MAE:0.7192)
